# Mandatory Assignment 3


## US Jobs and Import Competition from China 
Af: Mathias Nikolaisen(dnt357), Oliver Mattthews(jvc975) & Clara Maj Folkmann(djz279)

### Problem 1

In [5]:
import pandas as pd


data_path = 'A3_X2017S_MetricsI.dta'
df = pd.read_stata(data_path)

# Beregn beskrivende statistik for hver periode
summary_stats_period_0 = df[df['t2'] == 0][['dsL', 'dIPWusch', 'college', 'routine', 'dIPWusmx', 'foreignborn', 'dIPWotch', 'dIPWukch', 'dIPWotmx']].describe()
summary_stats_period_1 = df[df['t2'] == 1][['dsL', 'dIPWusch', 'college', 'routine', 'dIPWusmx', 'foreignborn', 'dIPWotch', 'dIPWukch', 'dIPWotmx']].describe()

# Udskriv summariske statistikker for begge perioder
print("1990-2000 (t2 = 0):")
print(summary_stats_period_0)

print("2000-2007 (t2 = 1):")
print(summary_stats_period_1)

1990-2000 (t2 = 0):
              dsL    dIPWusch     college  ...    dIPWotch    dIPWukch    dIPWotmx
count  662.000000  662.000000  662.000000  ...  662.000000  662.000000  662.000000
mean    -0.957021    1.186600   42.097832  ...    1.016592    0.728172    0.014953
std      2.240247    1.838066    8.530094  ...    1.083533    1.053982    0.112670
min    -19.167072   -0.075351   19.943975  ...   -0.467243   -0.687543   -1.519587
25%     -2.084500    0.248525   35.936215  ...    0.282036    0.122727    0.000772
50%     -0.685493    0.744347   42.515060  ...    0.663031    0.544912    0.012937
75%      0.379791    1.426852   47.567284  ...    1.398126    1.002490    0.037991
max      5.664484   25.405270   62.898552  ...    8.616317   13.057814    0.516513

[8 rows x 9 columns]
2000-2007 (t2 = 1):
              dsL    dIPWusch     college  ...    dIPWotch    dIPWukch    dIPWotmx
count  662.000000  662.000000  662.000000  ...  662.000000  662.000000  662.000000
mean    -2.338758    2.67

In [6]:
import statsmodels.api as sm

y = df['dsL']
X = df[['t2', 'dIPWusch', 'college', 'foreignborn', 'routine']]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    dsL   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     159.9
Date:                Wed, 23 Oct 2024   Prob (F-statistic):          6.08e-133
Time:                        09:00:05   Log-Likelihood:                -2788.1
No. Observations:                1324   AIC:                             5588.
Df Residuals:                    1318   BIC:                             5619.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           5.0983      0.544      9.371      

### Problem 2

In [35]:
from linearmodels.iv import IV2SLS

y = df['dsL']

X = df[['t2', 'dIPWusch', 'college', 'foreignborn', 'routine']]
Z = df['dIPWotch']
endog = df['dIPWotch']

X = sm.add_constant(X)

iv_model = IV2SLS(y, X, endog=endog, instruments=Z).fit()

print(iv_model.summary)


                          IV-2SLS Estimation Summary                          
Dep. Variable:                    dsL   R-squared:                      0.4058
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4030
No. Observations:                1324   F-statistic:                    618.48
Date:                Wed, Oct 23 2024   P-value (F-stat)                0.0000
Time:                        11:03:37   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
const           4.9502     0.4773     10.371     0.0000      4.0147      5.8857
t2             -1.1932     0.1438    -8.2955    

In [34]:
y = df['dsL']
X = df[['t2', 'college', 'foreignborn', 'routine']]
endog = df['dIPWusch']
Z = df[['dIPWotch', 'dIPWukch']]  

X = sm.add_constant(X)

iv_model = IV2SLS(y, X, endog=endog, instruments=Z).fit()
iv_resid = iv_model.resids
help_reg = sm.OLS(iv_resid, sm.add_constant(Z)).fit()
nR2 = help_reg.nobs * help_reg.rsquared

from scipy.stats import chi2
p_value = chi2.sf(nR2, df=Z.shape[1] - 1)

print(iv_model.summary)

print(f"Hansen's J-statistic: {nR2}")
print(f"P-value for J-statistic: {p_value}")


                          IV-2SLS Estimation Summary                          
Dep. Variable:                    dsL   R-squared:                      0.3426
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3401
No. Observations:                1324   F-statistic:                    584.90
Date:                Wed, Oct 23 2024   P-value (F-stat)                0.0000
Time:                        11:03:33   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
const           4.6095     0.5120     9.0030     0.0000      3.6060      5.6129
t2             -1.1333     0.1743    -6.5015    

In [38]:
y = df['dsL']  

X = df[['t2', 'college', 'foreignborn', 'routine']] 

endog = df[['dIPWusch', 'dIPWusmx']] 

Z = df[['dIPWotch', 'dIPWotmx']] 

X = sm.add_constant(X)

iv_model = IV2SLS(y, X, endog=endog, instruments=Z).fit()

print(iv_model.summary.as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &        dsL         & \textbf{  R-squared:         } &      0.3088      \\
\textbf{Estimator:}        &      IV-2SLS       & \textbf{  Adj. R-squared:    } &      0.3057      \\
\textbf{No. Observations:} &        1324        & \textbf{  F-statistic:       } &      538.01      \\
\textbf{Date:}             &  Wed, Oct 23 2024  & \textbf{  P-value (F-stat)   } &      0.0000      \\
\textbf{Time:}             &      11:07:58      & \textbf{  Distribution:      } &     chi2(6)      \\
\textbf{Cov. Estimator:}   &       robust       & \textbf{                     } &                  \\
\textbf{}                  &                    & \textbf{                     } &                  \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                     & \textbf{Parameter} & \textbf{Std. Err.} & \textbf{T-stat} & \textbf{P-value} & \textbf{Lower CI} & \textbf{Upper CI}  \\
\midrule
\textbf{const}       &       

### Problem 3

In [18]:
#3.2
df['t2_dIPWusch'] = df['t2'] * df['dIPWusch']
df['t2_dIPWotch'] = df['t2'] * df['dIPWotch']

df['fitted_dIPWusch'] = sm.OLS(df['dIPWusch'], sm.add_constant(df[['dIPWotch', 't2', 'college', 'foreignborn', 'routine']])).fit().fittedvalues
df['fitted_t2_dIPWusch'] = sm.OLS(df['t2_dIPWusch'], sm.add_constant(df[['t2_dIPWotch', 't2', 'college', 'foreignborn', 'routine']])).fit().fittedvalues

second_stage_model = sm.OLS(df['dsL'], sm.add_constant(df[['fitted_dIPWusch', 'fitted_t2_dIPWusch', 't2', 'college', 'foreignborn', 'routine']]))
second_stage_results = second_stage_model.fit()

second_stage_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dsL   R-squared:                       0.406
Model:                            OLS   Adj. R-squared:                  0.403
Method:                 Least Squares   F-statistic:                     150.2
Date:                Wed, 23 Oct 2024   Prob (F-statistic):          3.15e-145
Time:                        12:39:51   Log-Likelihood:                -2756.9
No. Observations:                1324   AIC:                             5528.
Df Residuals:                    1317   BIC:                             5564.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  4.4997      0.538      8.369      0.000       3.445       5.554
fitted_dIPWusch       -0.2877      0.092     -3.134      0.002      -0.468      -0.108
fitted_t2_dIPWusch    -0.1033      0.094     -1.094      0.274      -0.289       0.082
t2                    -0.9525      0.179     -5.335      0.000      -1.303      -0.602
college                0.0704      0.007     10.075      0.000       0.057       0.084
foreignborn           -0.0062      0.010     -0.648      0.517      -0.025       0.013
routine               -0.2847      0.020    -14.496      0.000      -0.323      -0.246
==============================================================================
Omnibus:                      139.333   Durbin-Watson:                   1.485
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              737.655
Skew:                          -0.329   Prob(JB):                    6.61e-161
Kurtosis:                       6.597   Cond. No.                         547.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
y = df['dsL']  
X = df[['t2', 'college', 'foreignborn', 'routine']] 
endog = df[['dIPWusch']]  
Z = df[['dIPWotch', 'dIPWotmx']] 
X = sm.add_constant(X)

iv_model = IV2SLS(y, X, endog=endog, instruments=Z).fit()

print(iv_model.summary)


                          IV-2SLS Estimation Summary                          
Dep. Variable:                    dsL   R-squared:                      0.3332
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3307
No. Observations:                1324   F-statistic:                    582.06
Date:                Wed, Oct 23 2024   P-value (F-stat)                0.0000
Time:                        12:47:35   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
const           4.5477     0.5407     8.4111     0.0000      3.4880      5.6074
t2             -1.0858     0.2012    -5.3954    

## Problem 4

## Problem 5